# Imports

In [1]:
import pathlib
import xarray as xr

## Open dataset

In [2]:
## Path to file server
SERVER_FP = pathlib.Path("/vortexfs1/share")

## Specify folder location for saving trimmed data ("./" means current directory)
save_fp = pathlib.Path("./")

## path to theta_o
theta_o_fp = SERVER_FP / pathlib.Path("cmip6/data/cmip6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Omon/thetao/gn/1")

## open
data = xr.open_mfdataset(theta_o_fp.glob("*.nc"))

/vortexfs1/home/kcarr/nashvar/envs/lib/python3.11/site-packages/xarray/conventions.py:287: SerializationWarning: variable 'thetao' has multiple fill values {1e+20, 1e+20} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


## Trim it

In [3]:
def trim(data, lat_range, lon_range=[0,360]):
    """select part of data in given longitude/latitude range"""

    ## helper function to check if 'x' is in 'x_range'
    isin_range = lambda x, x_range: (x_range[0] <= x) & (x <= x_range[1])
    
    ## get mask for data in given lon/lat range
    in_lon_range = isin_range(data['lon'],lon_range)
    in_lat_range = isin_range(data["lat"], lat_range)
    
    in_lonlat_range = in_lon_range & in_lat_range

    ## load to memory
    in_lonlat_range.load()

    ## Retain all points with at least one valid grid cell
    x_idx = in_lonlat_range.any("nlat")
    y_idx = in_lonlat_range.any("nlon")

    ## select given points
    return data.isel(nlon=x_idx, nlat=y_idx)

In [4]:
trim(data, lat_range=[-5,5], lon_range=[1,4])

<xarray.Dataset> Size: 54MB
Dimensions:    (time: 1980, lev: 60, nlat: 38, nlon: 3, d2: 2, vertices: 4)
Coordinates:
    lat        (nlat, nlon) float64 912B dask.array<chunksize=(38, 3), meta=np.ndarray>
  * lev        (lev) float64 480B 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    lon        (nlat, nlon) float64 912B dask.array<chunksize=(38, 3), meta=np.ndarray>
  * nlat       (nlat) int32 152B 169 170 171 172 173 174 ... 202 203 204 205 206
  * nlon       (nlon) int32 12B 37 38 39
  * time       (time) object 16kB 1850-01-15 13:00:00.000007 ... 2014-12-15 1...
Dimensions without coordinates: d2, vertices
Data variables:
    thetao     (time, lev, nlat, nlon) float32 54MB dask.array<chunksize=(1, 30, 38, 3), meta=np.ndarray>
    time_bnds  (time, d2) object 32kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    lat_bnds   (nlat, nlon, vertices) float32 2kB dask.array<chunksize=(38, 3, 4), meta=np.ndarray>
    lon_bnds   (nlat, nlon, vertices) float32 2kB dask.array<chunksize=(38, 3, 4), meta=np.ndarray>
    lev_bnds   (lev, d2) float32 480B dask.array<chunksize=(60, 2), meta=np.ndarray>
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    case_id:                15
    cesm_casename:          b.e21.BHIST.f09_g17.CMIP6-historical.001
    contact:                cesm_cmip6@ucar.edu
    creation_date:          2019-01-16T22:01:19Z
    ...                     ...
    sub_experiment:         none
    sub_experiment_id:      none
    branch_time_in_parent:  219000.0
    branch_time_in_child:   674885.0
    branch_method:          standard
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.NCAR.CESM2.h...